## Loading data

https://www.kaggle.com/competitions/tensorflow-speech-recognition-challenge

In [24]:
import os 

import librosa
from librosa.feature import melspectrogram
from librosa.display import specshow
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

### Setup

In [25]:
classes_known = ['up', 'down', 'left', 'right', 'on', 'off', 'yes', 'no']

# uncomment below line for less 'known' classes
# classes_known = ['yes', 'no']

In [26]:
raw_data_base_path = 'data/raw'
audio_base_path = f'{raw_data_base_path}/audio'
out_dir = f'data/processed{len(classes_known)}'

In [27]:
with open(f'{raw_data_base_path}/validation_list.txt', 'r') as file:
    validation_list = file.readlines()
    validation_list = {path.strip() for path in validation_list}

with open(f'{raw_data_base_path}/testing_list.txt', 'r') as file:
    testing_list = file.readlines()
    testing_list = {path.strip() for path in testing_list}

In [28]:
def generate_mel_spectogram(signal, sample_rate, folder: str, name: str):
    """
    name - np. "bed/00f0204f_nohash_0.wav"
    """
    mel_spectrogram = melspectrogram(y=signal, sr=sample_rate)
    mel_spectrogram_abs = np.abs(mel_spectrogram)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram_abs, ref=np.max)
    # flip so that low frequencies are at the bottom
    mel_spectrogram_db_flipped = np.flip(mel_spectrogram_db, axis=0)

    save_path = f'{out_dir}/{folder}/{name[:-4]}.png'
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.imsave(save_path, mel_spectrogram_db_flipped)

# Generate mel-spectrograms for known classes

In [29]:
datasets = ['train', 'valid', 'test']

In [30]:
# number of observations in all known classes combined (train/valid/test)
known_counts = [0, 0, 0]

for label in classes_known:
    print(f'Processing "{label}" class...')
    for filename in os.listdir(f'{audio_base_path}/{label}'):
        
        full_name = f'{label}/{filename}'
        signal, sample_rate = librosa.load(f'{audio_base_path}/{full_name}')

        # check which subset
        if full_name in validation_list:
            dataset_id = 1
        elif full_name in testing_list:
            dataset_id = 2
        else:
            dataset_id = 0

        folder = datasets[dataset_id]
        known_counts[dataset_id] += 1

        generate_mel_spectogram(signal, sample_rate, f'{folder}/known', full_name)

Processing "up" class...


Processing "down" class...
Processing "left" class...
Processing "right" class...
Processing "on" class...
Processing "off" class...
Processing "yes" class...
Processing "no" class...


# Generate mel-spectrograms for unknown class

In [31]:
known_counts

[14792, 2071, 2067]

In [32]:
unknown_classes_count = 30 - len(classes_known)

We want to create a varied unknown class, so we take the same number of samples from each unknown class

In [33]:
subset_size_per_class = [c // unknown_classes_count for c in known_counts]
subset_size_per_class

[672, 94, 93]

In [34]:
rng = np.random.default_rng(28)

for label in os.listdir(audio_base_path):
    if label in classes_known or label == '_background_noise_':
        continue

    path = f'{audio_base_path}/{label}'
    if not os.path.isdir(path):
        continue

    files = os.listdir(f'{audio_base_path}/{label}')
    # randomise files order
    rng.shuffle(files)
    # track the number of files we processed for this class
    curr_counts = [0, 0, 0]

    print(f'Processing "{label}" class...')
    for filename in files:
        
        full_name = f'{label}/{filename}'
        signal, sample_rate = librosa.load(f'{audio_base_path}/{full_name}')

        # check which subset
        if full_name in validation_list:
            dataset_id = 1
        elif full_name in testing_list:
            dataset_id = 2
        else:
            dataset_id = 0

        # if we have enough of this class in this dataset then move on to the next file
        if curr_counts[dataset_id] == subset_size_per_class[dataset_id]:
            continue 

        folder = datasets[dataset_id]
        curr_counts[dataset_id] += 1
        # here we merge class label with the filename
        generate_mel_spectogram(signal, sample_rate, f'{folder}/unknown', full_name.replace('/', ''))

Processing "marvin" class...


Processing "six" class...
Processing "sheila" class...
Processing "four" class...
Processing "go" class...
Processing "stop" class...
Processing "five" class...
Processing "seven" class...
Processing "bird" class...
Processing "bed" class...
Processing "nine" class...
Processing "two" class...
Processing "dog" class...
Processing "happy" class...
Processing "tree" class...
Processing "eight" class...
Processing "cat" class...
Processing "wow" class...
Processing "house" class...
Processing "three" class...
Processing "zero" class...
Processing "one" class...


# Generate mel-spectrogramns for silence class

We want silence files to have the same length as the other files. If a piece of audio at the end of the file is too short, in order to standardize the audio data, we will extend it with silence to ensure all files have equal duration of sound.

In [35]:
signal, _ = librosa.load(f'{audio_base_path}/bed/00f0204f_nohash_0.wav')
audio_len = len(signal)

In [36]:
valid_frac = known_counts[1] / np.sum(known_counts)
valid_frac

0.10940306391970417

In [37]:
# train, valid, test
silence_signals = [[], [], []]
bg_noise_path = f'{audio_base_path}/_background_noise_'

rng = np.random.default_rng(26)

for filename in os.listdir(bg_noise_path):
    if not filename.endswith('.wav'):
        continue

    full_silence_signal, sample_rate = librosa.load(f'{bg_noise_path}/{filename}')
    silence_segments = []
    # split into segments
    for boundary in range(0, len(full_silence_signal), audio_len):
        silence = full_silence_signal[boundary:boundary + audio_len]
        
        silence_len = len(silence)
        if silence_len < audio_len:
            # fill with 0s
            silence = np.pad(silence, (0, audio_len - silence_len), mode='constant')
        
        silence_segments.append(silence)
        
    # add new samples to the respective datasets
    rng.shuffle(silence_segments)
    valid_size = int(np.round(valid_frac * len(silence_segments)))
    # valid
    silence_signals[1].extend(silence_segments[:valid_size])
    # test
    silence_signals[2].extend(silence_segments[valid_size:2*valid_size])
    # train
    silence_signals[0].extend(silence_segments[2*valid_size:])

In [38]:
print('Train, valid, test')
[len(s) for s in silence_signals]

Train, valid, test


[310, 46, 46]

There are very little samples in the *silence* class compared to the other two classes. Below we generate additional samples in the following way:

1. Add reversed versions of each signal to the dataset
2. Make multiple copies of each signal

### 1. Add reversed versions of each signal to the dataset

In [39]:
for silence_dataset in silence_signals:
    for i in range(len(silence_dataset)):
        signal_original = silence_dataset[i]
        signal_reversed = np.flip(signal_original)
        silence_dataset.append(signal_reversed)

In [40]:
print('Train, valid, test')
[len(s) for s in silence_signals]

Train, valid, test


[620, 92, 92]

### Generate spectrograms

In [41]:
for dataset_name, silence_dataset in zip(datasets, silence_signals):
    for i, signal in enumerate(silence_dataset):
        generate_mel_spectogram(signal, sample_rate, f'{dataset_name}/silence', f'{i}.wav')

### 2. Make multiple copies of each signal

In [42]:
import shutil

In [43]:
copies_needed = int(known_counts[0] // len(silence_signals[0])) - 1
copies_needed

22

In [44]:
for dataset_name in datasets:
    silence_path = f'{out_dir}/{dataset_name}/silence'
    orig_files = os.listdir(silence_path)

    for filename in orig_files:
        file_idx = filename[:-4]
        for copy_id in range(copies_needed):
            shutil.copy(f'{silence_path}/{filename}', f'{silence_path}/{file_idx}_{copy_id}.png')

# Generate a small subset of data for testing purposes

In [45]:
def generate_subset(size: int):
    for label in ['known/yes', 'known/no', 'unknown', 'silence']:
        for dataset in datasets:
            processed = f'{out_dir}/{dataset}/{label}'
            subset = f'data/subset/{dataset}/{label}'
            os.makedirs(subset, exist_ok=True)
            n_files = 0
            for filename in os.listdir(processed):
                shutil.copy(f'{processed}/{filename}', f'{subset}/{filename}')
                n_files += 1
                if n_files == size:
                    break

In [46]:
# generate_subset(1)